# Python Microservices Development

## 1. Understanding Microservices

The microservices trend is one improvement that has emerged in the last few years, partially based on companies' willingness to speed up their release cycles.

### Origins of Service-Oriented Architecture
People often mention Service-Oriented Architecture (SOA) when they are trying to explain
what microservices are.

>SOA predates microservices, and its core principle is the idea that you organize applications into a discrete unit of functionality that can be accessed remotely and acted upon and updated independently. 
- Wikipedia

Each unit in this preceding definition is a self-contained service, which implements one facet of a business, and provides its feature through some interface.

SOA services could communicate via Inter-Process Communication (IPC) using sockets on the same machine, through shared memory, through indirect message queues, or even with Remote Procedure Calls (RPC).

However, it is common to say that **microservices are one specialization of SOA**

### The monolithic approach

The following points summarize the pros and cons of the monolithic approach:
- Starting a project as a monolith is easy, and probably the best approach.
- A centralized database simplifies the design and organization of the data.
- Deploying one application is simple.
- Any change in the code can impact unrelated features. When something breaks, the whole application may break.
- Solutions to scale your application are limited: you can deploy several instances, but if one particular feature inside the app takes all the resources, it impacts everything.
- As the code base grows, it's hard to keep it clean and under control.

There are, of course, some ways to avoid some of the issues described here. The obvious solution is to split the application into separate pieces, even if the resulting code is still going to run in a single process. Developers do this by building their apps with external libraries and frameworks. Those tools can be in-house or from the **Open Source Software (OSS)** community.


Building a web app in Python if you use a framework like Flask, lets you focus on the business logic, and makes it very appealing to externalize some of your code into Flask extensions and small Python packages. And splitting your code into small packages is often a good idea to control your application growth.

> "Small is beautiful."
- The UNIX Philosophy


### The microservice approach


The way woulb be organize the code into several separate components that run in separate processes. Instead of having a single application in charge of everything, we would split it into many different microservices.

A microservice is a lightweight application, which provides a narrowed list of features with a well-defined contract. It's a component with a single responsibility, which can be developed and deployed independently.

### Microservice benefits

While the microservices architecture looks more complicated than its monolithic counterpart, its advantages are multiple. It offers the following:

- Separation of concerns
- Smaller projects to deal with
- More scaling and deployment options

#### Separation of concerns

First of all, each microservice can be developed independently by a separate team. The team in charge can make it in **whatever programming language and database**, as long as it has a **well-
documented HTTP API**.

This loose coupling improves the overall project velocity a lot, as we apply, at the service level, a philosophy similar to the _single responsibility_ principle. The single responsibility principle was defined by Robert Martin to explain that a class
should have **only one reason to change**; in other words, each class should provide a **single, well-defined feature**. Applied to microservices, it means that we want to make sure that
**each microservice focuses on a single role**.

#### Smaller projects

The second benefit is breaking the complexity of the project. When you add a feature to an application such as PDF reporting, even if you do it cleanly, you make the base code bigger,
more complicated, and sometimes, slower. Building that feature in a separate application avoids this problem, and makes it easier to write it with whatever tools you want. You can refactor it often, shorten your release cycles, and stay on top of things. The growth of the application remains under your control. Dealing with a smaller project also reduces risks when improving the application.

#### Scaling and deployment

Finally, having your application split into components makes it easier to scale depending on your constraints.

We can, thus, summarize the benefits of microservices as follows:
- A team can develop each microservice independently, and use whatever technological stack makes sense. They can define a custom release cycle. All they need to define is a language-agnostic HTTP API.
- Developers break the application complexity into logical components. Each microservice focuses on doing one thing well.
- Since microservices are standalone applications, there's a finer control on deployments, which makes scaling easier.


### Microservices pitfalls

You need to be aware of these main problems you might have to deal with when coding microservices:
- Illogical splitting
- More network interactions
- Data storing and sharing
- Compatibility issues
- Testing

#### Illogical splitting

The first issue of a microservice architecture is how it gets designed. There's no way a team can come up with the perfect microservice architecture in the first shot. The design needs to mature with some try-and-fail cycles. And adding and removing
microservices can be more painful than refactoring a monolithic application.

You can mitigate this problem by avoiding splitting your app in microservices if the split is
not evident.

> _Premature splitting is the root of all evil_

For instance, if you always have to deploy two microservices together, or if one change in a microservice impacts the data model of another one, the odds are that you did not split the
application correctly, and that those two services should be reunited.

#### More network interactions

The second problem is the amount of network interactions added to build the same application. In the monolithic version, even if the code gets messy, everything happens in the same process, and you can send back the result without having to call too many
backend services to build the actual response.

#### Data storing and sharing
Another problem is data storing and sharing. An effective microservice needs to be independent of other microservices, and **ideally, should not share a database**.

>Avoiding data duplication as much as possible while keeping
microservices in isolation is one of the biggest challenges in designing microservices-based applications.



#### Compatibility issues

Another problem happens when a feature change impacts several microservices. If a change affects in a backward incompatible way the data that travels between services, you're in for
some trouble. A **good versioning and API** design hygiene help to mitigate those issues.

#### Testing

Last, when you want to do some end-to-end tests and deploy your whole app, you now have to deal with many bricks. You need to have a robust and agile deployment process to be efficient. You need to be able to play with your whole application when you develop it. You can't fully test things out with just one piece of the puzzle.

The **pitfalls of using microservices** can be summarized as follows:
- Premature splitting of an application into microservices can lead to architectural problems
- Network interactions between microservices add weaknesses spots and additional overhead
- Testing and deploying microservices can be complex
- And the biggest challenge--data sharing between microservices is hard

### Implementing microservices with Python

#### The WSGI standard

What strikes most web developers who start with Python is how easy it is to get a web application up and running.

The Python web community has created a standard called **Web Server Gateway Interface (WSGI)**. It simplifies a lot how you
can write a Python application in order to serve HTTP requests.
When your code uses that standard, your project can be executed by standard web servers like Apache or nginx, using WSGI extensions like uwsgi or mod_wsgi .

Your application just has to deal with incoming requests and send back JSON responses, and Python includes all that goodness in its standard library.

You can create a fully functional microservice that returns the server's local time with a vanilla Python module of fewer than 10 lines.

In [2]:
import json
import time

def application(environ, start_response):
    headers = [('Content-type', 'application/json')]
    start_response('200 OK', headers)
    return [bytes(json.dumps({'time': time.time()}), 'utf8')]

The biggest problem with WSGI though is its synchronous nature. The application function you saw in the preceding code is called exactly once per incoming request, and when the function returns, it has to send back the response. That means that every time you call the function, it will block until the response is ready. 

WSGI servers will let you run a pool of threads to serve several requests concurrently. But you can't run thousands of them, and as soon as the pool is exhausted, the next request will block the client's access even if your microservice is doing nothing but idling and waiting for backend services' responses. That's one of the reasons why non-WSGI frameworks like Twisted and Tornado, and in JavaScript land, Node.js, became very successful--it's fully async.

There's, however, one trick to boost synchronous web applications--Greenlet, which is explained in the following section.

#### Greenlet and Gevent

The general principle of asynchronous programming is that the process deals with several concurrent execution contexts to simulate parallelism.

Greenlets are pseudo-threads that are very cheap to instantiate, unlike real threads, and that can be used to call Python functions. Within those functions, you can switch, and give back
the control to another function. The switching is done with an event loop, and allows you to write an asynchronous application using a thread-like interface paradigm.

The Gevent project is built on top of Greenlet, and offers an
implicit and automatic way of switching between greenlets, among many other things.

#### asyncio

The asyncio module was born out of that side project and
added into Python. The way the Python core developers coded asyncio, and how they elegantly extended the language with the async and await keywords to implement coroutines, made asynchronous applications, the code looks very elegant and close to synchronous programming. By doing this, Python did a great job at avoiding the callback syntax mess we sometimes see
in Node.js applications.

KeepSafe's aiohttp is one of them, and building the same microservice, fully asynchronous, with it would simply need these few elegant lines:

```python
from aiohttp import web
import time
async def handle(request):
    return web.json_response({'time': time.time()})
if __name__ == '__main__':
    app = web.Application()
    app.router.add_get('/', handle)
    web.run_app(app)
```

## 2. Discovering Flask

The Flask framework will only try to make sure it has enough hooks to be extended by external libraries to provide all kinds of features. In other words, using SQLAlchemy in Flask, and making sure you're doing the right thing with SQL sessions and transactions, will mostly consist of adding a package like Flask-SQLAlchemy in your project. And if you don't like how that particular library integrates SLQAlchemy, you're free to use another one, or to build your integration.

Of course, that's not a silver bullet. Being completely free in your choices also means it's easier to make poor decisions, and build an application that relies on defective libraries or one that's not well designed.

### How Flask handles requests
The framework entry point is the Flask class in the flask.app module. Running a Flask application means running one single instance of this class, which will take care of handling incoming **Web Server Gateway Interface (WSGI)** requests, dispatch them to the right code, and then return a response.

Here's a very basic example of a fully functional Flask application:


```python
from flask import Flask, jsonify

app = Flask(__name__)

@app.route('/api')
def my_microservice():
    return jsonify({'Hello': 'World!'})

if __name__ == '__main__':
    app.run()
```

#### Routing

The routing happens in `app.url_map`, which is an instance of Werkzeug's Map class. That class uses regular expressions to determine if a function decorated by `@app.route` matches
the incoming request. The routing only looks at the path you provided in the route call to see if it matches the client's request. By default, the mapper will only accept GET , OPTIONS , and HEAD calls on a declared route.

To add support specific methods:

```python
@app.route('/api', methods=['POST', 'DELETE', 'GET'])
def my_microservice():
    return jsonify({'Hello': 'World!'})
```

You can use variables using the <VARIABLE_NAME> syntax. This notation is pretty standard and allows you to describe endpoints with dynamic values.

```python
@app.route('/api/person/<int:person_id>')
def person(person_id):
    response = jsonify({'Hello': person_id})
    return response

$ curl localhost:5000/api/person/3
{
    "Hello": "3"
}
```

#### Request

When a request comes in, Flask calls the view inside a thread-safe block, and uses **Werzeug's local helper**. This helper
does a job similar to Python's **threading.local** and makes
sure that each thread has an isolated environment, specific to that request.

In other words, when you access the global request object in your view, you are guaranteed that it's unique to your thread, and will not leak data to another thread in a multi-threaded
environment.

#### Response 

## Designing Runnerly

